In [9]:
import requests
import re

urls = [
    'https://skjema.geonorge.no/SOSI/produktspesifikasjon/Matrikkelen-Adresse/20200501/matrikkelenAdresse.xsd',
    'https://skjema.geonorge.no/SOSI/produktspesifikasjon/AR50/20220523/ar50.xsd'
]

def get_xml_documentation(url: str) -> bool:
    res = requests.get(url)
    dataset_name = re.search(r'produktspesifikasjon/([^/]+)', url).group(1)
    filepath = f'./xml_docs/{dataset_name}_documentation.xml'
    print(filepath)

    if not res.status_code  == 200:
        raise RuntimeError('Could not load XML schema for {filename}')

    with open(filepath, 'wb') as file:
        file.write(res.content)

    return True


for url in urls: 
    get_xml_documentation(url)

./xml_docs/Matrikkelen-Adresse_documentation.xml
./xml_docs/AR50_documentation.xml


In [42]:
from langchain_community.document_loaders import UnstructuredXMLLoader
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter

loader = UnstructuredXMLLoader('./xml_docs/AR50_documentation.xml', mode='elements', xml_keep_tags=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False
)

docs_splitted = text_splitter.split_documents(docs)

docs_splitted[9].page_content

'-- Definition -- \r\narea with at least 60 trees per hectare which are or  may become 5 metres tall, and which should be evenly distributed over the area</documentation>\r\n        </annotation>\r\n      </enumeration>\r\n      <enumeration value="50">\r\n        <annotation>\r\n          <documentation>Open firm ground: fastmark som ikke er jordbruksareal, skog, bebygd eller samferdsel'

In [105]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from langchain_core.documents import Document


xml_path = './xml_docs/AR50_documentation.xml'


tree = ET.parse(xml_path)
root = tree.getroot()

# Iterate through each child of the <schema> element
docs = [Document(page_content=parseString(ET.tostring(child)
                                          ).toprettyxml(newl="")) for child in root]

docs

[Document(page_content='<?xml version="1.0" ?><xs:annotation xmlns:xs="http://www.w3.org/2001/XMLSchema">\t\n    \t<xs:documentation>Denne spesifikasjonen beskriver datasettet AR50 – Arealressurskart for målestokk 1:50 000. Dette er et landsdekkende datasett som viser hovedtyper av arealressurser i målestokk 1:50 000 tilpasset bruk i målestokker fra 1:20 000 til 1:100 000. AR50 fremstilt ved en generalisering av FKB-AR5 under tregrensa og tolking av satellittbilder over tregrensa. For klassifisering av treslag har det tillegg har det vært benyttet N50 skog uten registrert treslag der det ikke finnes AR5. Figurer som er mindre enn 15 dekar vises ikke på kartet, men blir slått sammen med tilgrensende områder.</xs:documentation>\t\n  </xs:annotation>'),
 Document(page_content='<?xml version="1.0" ?><xs:import xmlns:xs="http://www.w3.org/2001/XMLSchema" namespace="http://www.opengis.net/gml/3.2" schemaLocation="http://schemas.opengis.net/gml/3.2.1/gml.xsd"/>'),
 Document(page_content='<?xm

In [104]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv('../../.env')

faiss_index = FAISS.from_documents(docs, OpenAIEmbeddings())
docs = faiss_index.similarity_search("fulldyrka jord", k=3)
for doc in docs:
    print(doc.page_content + f'\n{"-"*100}\n')

<?xml version="1.0" ?><xs:simpleType xmlns:xs="http://www.w3.org/2001/XMLSchema" name="ArealressursVegetasjonsdekkeEnumerationType">	
    	<xs:annotation>		
      		<xs:documentation>AreaResourceVegetationCover: informasjon om vegetasjonsdekkets frodighet på snaumark


-- Definition - - 
information concerning the lushness of the vegetation cover on unforested land</xs:documentation>		
    	</xs:annotation>	
    	<xs:restriction base="string">		
      		<xs:enumeration value="51">			
        			<xs:annotation>				
          				<xs:documentation>Impediment: ikke vegetasjonsdekt

-- Definition -- 
No vegetation</xs:documentation>				
        			</xs:annotation>			
      		</xs:enumeration>		
      		<xs:enumeration value="52">			
        			<xs:annotation>				
          				<xs:documentation>Barren non-continuous: skrint til usammenhengende vegetasjonsdekke

-- Definition -- 
barren to non-continuous vegetation cover</xs:documentation>				
        			</xs:annotation>			
      		</xs:enu